In [7]:
from feast import FeatureStore
import pandas as pd
from joblib import load

In [8]:
# Getting our FeatureStore
store = FeatureStore(repo_path=".")

In [9]:
feature_service = store.get_feature_service("user_activity")

In [10]:
# Getting the latest features - we request the features data for two rows, 1085 (daily hours) and 1 (night hours)
features = store.get_online_features(
    features=feature_service,
    entity_rows=[{"USER_ID": 1}, {"USER_ID": 1085}]
).to_dict()
# get_online_features returns an OnlineResponse object - use to_dict to convert the feature values to a Python dictionary

In [11]:
# Converting the features to a DataFrame
features_df = pd.DataFrame.from_dict(data=features)

features_df.head()

,USER_ID,Age,Gender,Balance,CreditScore,EstimatedSalary,Tenure,NumOfProducts,HasCrCard,IsActiveMember,Geography_France,Geography_Spain,Geography_Germany
0,1,42,0,0.0,619,101348.88,2,1,1,1,True,False,False
1,1085,24,1,0.0,796,61349.37,8,2,1,0,True,False,False


In [12]:
# Carica il modello salvato
regression = load(r"models/logistic_regression.joblib")

In [13]:
columns_order = load(r'data/columns_order.joblib')
columns_order

FileNotFoundError: [Errno 2] No such file or directory: 'models/columns_order.joblib'

In [ ]:
# Rimuovi la colonna "USER_ID" e riordina le colonne secondo "columns_order"
features_df.drop('USER_ID', axis=1)
features_df = features_df[columns_order]

In [ ]:
# Verifica dei valori nulli
null_counts = features_df.isnull().sum()
null_counts

HasCrCard            0
Geography_Germany    0
Gender               0
Geography_France     0
CreditScore          0
Geography_Spain      0
NumOfProducts        0
IsActiveMember       0
Tenure               0
Age                  0
Balance              0
EstimatedSalary      0
dtype: int64

In [ ]:
# Fai le predizioni utilizzando il modello caricato
predictions = regression.predict(features_df)

# Stampa o ritorna le predizioni
predictions

array([0, 0], dtype=int64)

In [ ]:
# Carica il modello salvato
forest = load(r"data/random_forest_model.joblib")

# Fai le predizioni utilizzando il modello caricato
predictions = forest.predict(features_df)

# Stampa o ritorna le predizioni
predictions

array([1, 0], dtype=int64)